<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/SMOTE_schiz_models_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 7.5 MB/s eta 0:00:00


In [3]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [5]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

In [7]:
from imblearn.over_sampling import SMOTE

**Non-binary**

All features

In [8]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [9]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.9592592592592593
test score: 0.9888888888888889


In [10]:
params = opt.best_params_
svm1 = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'])
svm1.fit(X_train, y_train)
y_pred = svm1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        55
           1       1.00      0.97      0.98        60
           2       1.00      1.00      1.00        65

    accuracy                           0.99       180
   macro avg       0.99      0.99      0.99       180
weighted avg       0.99      0.99      0.99       180



In [11]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(svm1, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(svm1, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.9844444444444445 ± 0.02819683897877671
recall 0.9076066217732885 ± 0.1728740329334714


Only1D

In [12]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [13]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.9185185185185185
test score: 0.9111111111111111


In [14]:
params = opt.best_params_
svm2 = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'])
svm2.fit(X_train, y_train)
y_pred = svm2.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.75      0.84        55
           1       0.82      0.97      0.89        60
           2       0.98      1.00      0.99        65

    accuracy                           0.91       180
   macro avg       0.92      0.90      0.90       180
weighted avg       0.92      0.91      0.91       180



In [15]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(svm2, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(svm2, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.9422222222222223 ± 0.07646511348482335
recall 0.8217843467353271 ± 0.263689297774849


**Binary**

All features

In [16]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [17]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.9722222222222222
test score: 0.925


In [18]:
params = opt.best_params_
svm3 = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'])
svm3.fit(X_train, y_train)
y_pred = svm3.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.98      0.92        55
           1       0.98      0.88      0.93        65

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.92       120
weighted avg       0.93      0.93      0.93       120



In [19]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(svm3, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(svm3, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.9733333333333333 ± 0.03265986323710904
recall 0.8344444444444445 ± 0.22744962812309308


Only 1d-features

In [20]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [21]:
warnings.filterwarnings('ignore')
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.8944444444444444
test score: 0.8833333333333333


In [22]:
params = opt.best_params_
svm4 = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'])
svm4.fit(X_train, y_train)
y_pred = svm4.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.75      0.85        55
           1       0.82      1.00      0.90        65

    accuracy                           0.88       120
   macro avg       0.91      0.87      0.88       120
weighted avg       0.90      0.88      0.88       120



In [23]:
kf = KFold(n_splits=10)

crossval_accuracy = cross_val_score(svm4, X, y, scoring='accuracy', cv = kf)
mean_score = crossval_accuracy.mean()
std_score = crossval_accuracy.std()
print('accuracy', mean_score, '±', std_score)

crossval_recall = cross_val_score(svm4, X, y, scoring='recall_macro', cv = kf)
mean_score = crossval_recall.mean()
std_score = crossval_recall.std()
print('recall', mean_score, '±', std_score)

accuracy 0.9266666666666667 ± 0.08137703743822469
recall 0.8020548497680984 ± 0.2188994653344027


In [ ]:
import matplotlib.pyplot as plt

coefs = pd.DataFrame(
   svm4.coef_.T,
   columns=['Coefficients'], index=X_train.columns
)

coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)